# 09 - Hyperparameter Tuning

## Purpose

This notebook performs hyperparameter optimization for the Logistic Regression model using a two-stage approach: RandomizedSearchCV for broad exploration followed by GridSearchCV for fine-tuning.

## Configuration

| Parameter | Value |
|-----------|-------|
| Model | Logistic Regression |
| Scoring metric | ROC-AUC |
| Cross-validation | Stratified 5-fold |
| Tuning strategy | RandomizedSearch → GridSearch |

---
## 1. Setup & Imports

In [1]:
import numpy as np
import pandas as pd
import joblib
import warnings
from pathlib import Path
import os

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    RandomizedSearchCV,
    GridSearchCV,
    StratifiedKFold
)
from sklearn.metrics import (
    roc_auc_score,
    precision_recall_curve,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score
)

import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Setup complete.")

Setup complete.


---
## 2. Configuration

In [2]:
# Cross-validation configuration
CV_FOLDS = 5
SCORING = "roc_auc"

# RandomizedSearch configuration
N_ITER_RANDOM = 30

print(f"Scoring metric: {SCORING}")
print(f"CV folds: {CV_FOLDS}")
print(f"RandomizedSearch iterations: {N_ITER_RANDOM}")

Scoring metric: roc_auc
CV folds: 5
RandomizedSearch iterations: 30


---
## 3. Data Loading

In [3]:
PROJECT_ROOT = Path(os.getcwd()).parent
DATA_PROCESSED_PATH = PROJECT_ROOT / "data" / "processed"
MODELS_PATH = PROJECT_ROOT / "models"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_PROCESSED_PATH exists:", DATA_PROCESSED_PATH.exists())
print("MODELS_PATH exists:", MODELS_PATH.exists())

PROJECT_ROOT: /Users/omarpiro/churn_ml_decision
DATA_PROCESSED_PATH exists: True
MODELS_PATH exists: True


In [4]:
# Load processed datasets
X_train = np.load(DATA_PROCESSED_PATH / "X_train_processed.npy")
X_val = np.load(DATA_PROCESSED_PATH / "X_val_processed.npy")
y_train = np.load(DATA_PROCESSED_PATH / "y_train.npy")
y_val = np.load(DATA_PROCESSED_PATH / "y_val.npy")

# Load feature names for reference
feature_names = pd.read_csv(MODELS_PATH / "final_feature_names.csv")["feature_name"].tolist()

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"\nNumber of features: {len(feature_names)}")

X_train shape: (4225, 31)
X_val shape: (1409, 31)
y_train shape: (4225,)
y_val shape: (1409,)

Number of features: 31


In [5]:
# Check class distribution
train_churn_rate = y_train.mean()
val_churn_rate = y_val.mean()

print(f"Training set churn rate: {train_churn_rate:.2%}")
print(f"Validation set churn rate: {val_churn_rate:.2%}")

Training set churn rate: 26.53%
Validation set churn rate: 26.54%


---
## 4. Cross-Validation Setup

In [6]:
# Stratified K-Fold to maintain class distribution
cv_strategy = StratifiedKFold(
    n_splits=CV_FOLDS,
    shuffle=True,
    random_state=RANDOM_STATE
)

print(f"Cross-validation strategy: StratifiedKFold")
print(f"  - n_splits: {CV_FOLDS}")
print(f"  - shuffle: True")
print(f"  - random_state: {RANDOM_STATE}")

Cross-validation strategy: StratifiedKFold
  - n_splits: 5
  - shuffle: True
  - random_state: 42


---
## 5. Stage 1: RandomizedSearchCV (Exploration)

In [7]:
# Define broad parameter space for exploration
param_distributions = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "penalty": ["l1", "l2"],
    "solver": ["liblinear", "saga"],
    "class_weight": [None, "balanced"]
}

# Calculate total combinations
total_combinations = 1
for param, values in param_distributions.items():
    total_combinations *= len(values)
    print(f"  {param}: {len(values)} values")

print(f"\nTotal possible combinations: {total_combinations}")
print(f"RandomizedSearch will sample: {N_ITER_RANDOM} combinations")

  C: 6 values
  penalty: 2 values
  solver: 2 values
  class_weight: 2 values

Total possible combinations: 48
RandomizedSearch will sample: 30 combinations


In [8]:
# Base model
base_model = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=1000
)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=N_ITER_RANDOM,
    scoring=SCORING,
    cv=cv_strategy,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

print("RandomizedSearchCV configured.")

RandomizedSearchCV configured.


In [9]:
# Fit RandomizedSearchCV
print("Starting RandomizedSearchCV...")
random_search.fit(X_train, y_train)
print("\nRandomizedSearchCV complete.")

Starting RandomizedSearchCV...
Fitting 5 folds for each of 30 candidates, totalling 150 fits

RandomizedSearchCV complete.


In [10]:
# RandomizedSearch results
print("RandomizedSearchCV Results")
print("=" * 50)
print(f"Best CV score ({SCORING}): {random_search.best_score_:.4f}")
print(f"\nBest parameters:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

RandomizedSearchCV Results
Best CV score (roc_auc): 0.8453

Best parameters:
  solver: saga
  penalty: l1
  class_weight: None
  C: 1


In [11]:
# Top 5 configurations from RandomizedSearch
random_results = pd.DataFrame(random_search.cv_results_)
top_configs = random_results.nsmallest(5, "rank_test_score")[
    ["params", "mean_test_score", "std_test_score", "mean_train_score", "rank_test_score"]
].reset_index(drop=True)

print("\nTop 5 configurations from RandomizedSearch:")
for idx, row in top_configs.iterrows():
    print(f"\n#{idx+1} (Rank {row['rank_test_score']})")
    print(f"  CV Score: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f})")
    print(f"  Train Score: {row['mean_train_score']:.4f}")
    print(f"  Params: {row['params']}")


Top 5 configurations from RandomizedSearch:

#1 (Rank 1)
  CV Score: 0.8453 (+/- 0.0171)
  Train Score: 0.8496
  Params: {'solver': 'saga', 'penalty': 'l1', 'class_weight': None, 'C': 1}

#2 (Rank 2)
  CV Score: 0.8453 (+/- 0.0172)
  Train Score: 0.8496
  Params: {'solver': 'liblinear', 'penalty': 'l1', 'class_weight': None, 'C': 1}

#3 (Rank 3)
  CV Score: 0.8451 (+/- 0.0174)
  Train Score: 0.8497
  Params: {'solver': 'saga', 'penalty': 'l1', 'class_weight': None, 'C': 10}

#4 (Rank 4)
  CV Score: 0.8451 (+/- 0.0175)
  Train Score: 0.8496
  Params: {'solver': 'saga', 'penalty': 'l1', 'class_weight': 'balanced', 'C': 1}

#5 (Rank 5)
  CV Score: 0.8451 (+/- 0.0172)
  Train Score: 0.8496
  Params: {'solver': 'liblinear', 'penalty': 'l2', 'class_weight': None, 'C': 1}


---
## 6. Stage 2: GridSearchCV (Fine-tuning)

In [13]:
# Extract best parameters from RandomizedSearch
best_C = random_search.best_params_["C"]
best_penalty = random_search.best_params_["penalty"]
best_solver = random_search.best_params_["solver"]
best_class_weight = random_search.best_params_["class_weight"]

# Define refined grid around best C value
if best_C <= 0.01:
    C_range = [0.001, 0.005, 0.01, 0.05, 0.1]
elif best_C >= 100:
    C_range = [10, 50, 100, 500, 1000]
else:
    C_range = [best_C / 10, best_C / 2, best_C, best_C * 2, best_C * 10]

# Refined parameter grid
param_grid_refined = {
    "C": C_range,
    "penalty": [best_penalty],
    "solver": [best_solver],
    "class_weight": [best_class_weight]
}

print("Refined parameter grid for GridSearchCV:")
for param, values in param_grid_refined.items():
    print(f"  {param}: {values}")

Refined parameter grid for GridSearchCV:
  C: [0.1, 0.5, 1, 2, 10]
  penalty: ['l1']
  solver: ['saga']
  class_weight: [None]


In [14]:
# GridSearchCV for fine-tuning
grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    param_grid=param_grid_refined,
    scoring=SCORING,
    cv=cv_strategy,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

print("\nStarting GridSearchCV...")
grid_search.fit(X_train, y_train)
print("\nGridSearchCV complete.")


Starting GridSearchCV...
Fitting 5 folds for each of 5 candidates, totalling 25 fits

GridSearchCV complete.


In [15]:
# GridSearch results
print("GridSearchCV Results")
print("=" * 50)
print(f"Best CV score ({SCORING}): {grid_search.best_score_:.4f}")
print(f"\nBest parameters:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

GridSearchCV Results
Best CV score (roc_auc): 0.8453

Best parameters:
  C: 1
  class_weight: None
  penalty: l1
  solver: saga


In [16]:
# Compare RandomizedSearch vs GridSearch
print("\nComparison: RandomizedSearch vs GridSearch")
print("=" * 50)
print(f"RandomizedSearch best CV score: {random_search.best_score_:.4f}")
print(f"GridSearch best CV score:       {grid_search.best_score_:.4f}")
print(f"Improvement: {(grid_search.best_score_ - random_search.best_score_) * 100:.2f}%")


Comparison: RandomizedSearch vs GridSearch
RandomizedSearch best CV score: 0.8453
GridSearch best CV score:       0.8453
Improvement: 0.00%


---
## 7. Best Model Evaluation on Validation Set

In [17]:
# Get best model from GridSearch
best_model = grid_search.best_estimator_

print("Best Model Configuration:")
print(f"  C: {best_model.C}")
print(f"  penalty: {best_model.penalty}")
print(f"  solver: {best_model.solver}")
print(f"  class_weight: {best_model.class_weight}")

Best Model Configuration:
  C: 1
  penalty: l1
  solver: saga
  class_weight: None


In [18]:
# Predict probabilities on validation set
y_val_proba = best_model.predict_proba(X_val)[:, 1]
y_val_pred = best_model.predict(X_val)

# Evaluation with default threshold (0.5)
val_roc_auc = roc_auc_score(y_val, y_val_proba)

print("Validation Set Performance (threshold = 0.5)")
print("=" * 50)
print(f"ROC-AUC: {val_roc_auc:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=["No Churn", "Churn"]))

Validation Set Performance (threshold = 0.5)
ROC-AUC: 0.8585

Classification Report:
              precision    recall  f1-score   support

    No Churn       0.84      0.91      0.88      1035
       Churn       0.68      0.53      0.60       374

    accuracy                           0.81      1409
   macro avg       0.76      0.72      0.74      1409
weighted avg       0.80      0.81      0.80      1409



In [19]:
# Confusion matrix
cm = confusion_matrix(y_val, y_val_pred)
print("Confusion Matrix (threshold = 0.5):")
print(f"                 Predicted")
print(f"                 No Churn  Churn")
print(f"Actual No Churn    {cm[0,0]:5d}   {cm[0,1]:5d}")
print(f"Actual Churn       {cm[1,0]:5d}   {cm[1,1]:5d}")

Confusion Matrix (threshold = 0.5):
                 Predicted
                 No Churn  Churn
Actual No Churn      943      92
Actual Churn         176     198


---
## 8. Save best model

In [20]:
# Save best model
joblib.dump(best_model, MODELS_PATH / "best_model_tuned.joblib")
print(f"Best model saved: {MODELS_PATH / 'best_model_tuned.joblib'}")


Best model saved: /Users/omarpiro/churn_ml_decision/models/best_model.joblib


In [21]:
# Save tuning results
tuning_results = {
    "best_params": grid_search.best_params_,
    "best_cv_score": grid_search.best_score_,
    "cv_strategy": "StratifiedKFold",
    "cv_folds": CV_FOLDS,
    "scoring": SCORING,
    "random_state": RANDOM_STATE
}

joblib.dump(tuning_results, MODELS_PATH / "tuning_results.joblib")
print(f"Tuning results saved: {MODELS_PATH / 'tuning_results.joblib'}")

Tuning results saved: /Users/omarpiro/churn_ml_decision/models/tuning_results.joblib


In [22]:
# Summary of all saved artifacts
print("\n" + "=" * 60)
print("ARTIFACTS SUMMARY")
print("=" * 60)

print(f"\n{MODELS_PATH}/")
print(" ├─ best_model_tuned.joblib        (Tuned Logistic Regression)")
print(" └─ tuning_results.joblib    (Best hyperparameters & CV score)")



ARTIFACTS SUMMARY

/Users/omarpiro/churn_ml_decision/models/
 ├─ best_model.joblib        (Tuned Logistic Regression)
 └─ tuning_results.joblib    (Best hyperparameters & CV score)


---
## 9. Conclusion

This notebook focused exclusively on hyperparameter tuning for the
Logistic Regression model using cross-validation.

- Hyperparameters were optimized using ROC-AUC as the primary metric
- The best model configuration was selected and evaluated on the validation set
- No decision threshold optimization was performed at this stage

The tuned model and tuning metadata are saved and will be reused in the
next notebook dedicated to business-driven threshold optimization.